In [1]:
import os
import time
import copy
import numpy as n
import numpy.lib.recfunctions as rfn
from matplotlib import pyplot as plt

from sklearn.metrics import confusion_matrix

import torch
from torch import nn

## Functions
#### Load data files
`data_root` should contain the root directory of the folder downloaded from Dropbox.

In [2]:
def load_data(data_root, dlc_dir, ann_dir, verbose=False):
    
    dlc_path = os.path.join(data_root, dlc_dir)
    ann_path = os.path.join(data_root, ann_dir)
    all_data = {}
    if verbose: print("Loading files: ")
    for f_name in os.listdir(dlc_path):
        if f_name[-3:] != 'npy':
            continue

        dlc_file=os.path.join(dlc_path, f_name)
        ann_file=os.path.join(ann_path, 'Annotated_' + f_name)
        if verbose: print("\t" + f_name + "\n\tAnnotated_" + f_name)
        data_dlc = n.load(dlc_file)
        data_ann = n.load(ann_file)
        labels = data_dlc[0]
        dtype = [('t', n.int), ('ann', 'U30')]
        i = 0
        for label in data_dlc[0]:
            i += 1
            coord = 'x' if i % 2 == 0 else 'y'
            dtype += [(label + '_' + coord , n.float32 )]

        data_concat = n.concatenate((data_ann, data_dlc[1:]),axis=1)
        data = n.array(n.zeros(data_concat.shape[0]), dtype = dtype)
        for i in range(data_concat.shape[1]):
            data[dtype[i][0]] = data_concat[:, i]
        all_data[f_name[:-4]] = data
    return all_data

#### Velocities

In [3]:
def dot(a, b):
    return n.sum(a * b, axis=-1)

def mag(a):
    return n.sqrt(n.sum(a*a, axis=-1))

def get_angle(a, b):
    cosab = dot(a, b) / (mag(a) * mag(b)) # cosine of angle between vectors
    angle = n.arccos(cosab) # what you currently have (absolute angle)

    b_t = b[:,[1,0]] * [1, -1] # perpendicular of b

    is_cc = dot(a, b_t) < 0

    # invert the angles for counter-clockwise rotations
    angle[is_cc] = 2*n.pi - angle[is_cc]
    return 360 - n.rad2deg(angle)

In [4]:
def calculate_velocity(trial):
    names = []; dtypes = []; datas = []
    velocities_calculated = []
    for label in trial.dtype.names:
        if label[-2:] in ['_x', '_y']:
            names.append(label+'_vel')  
            dtypes += [n.float]
            datas += [n.zeros(trial.shape[0])]
            velocities_calculated.append(label)
    trial = rfn.append_fields(trial, names, datas, dtypes)
    trial = n.array(trial, trial.dtype)
    for label in velocities_calculated:
        vel = n.gradient(trial[label])
        trial[label + '_vel'] = vel
    return trial
def normalize_trial(trial, feature_labels, nan = -10000):
    ref_x = trial[feature_labels[1]].copy()
    ref_y = trial[feature_labels[0]].copy()
    for i,label in enumerate(feature_labels):
        if label[-1] == 'y':
    #         print('y-pre:',n.nanmax(features[:,i]))
            trial[label] -= ref_y
    #         print('y-post:', n.nanmax(features[:,i]))
        elif label[-1] == 'x':
    #         print('x-pre:',n.nanmax(features[:,i]))
            trial[label] -= ref_x
    #         print('x-post:', n.nanmax(features[:,i]))

    mouse_1_pos_labels = []
    mouse_2_pos_labels = []
    mouse_1_vel_labels = []
    mouse_2_vel_labels = []
    for label in feature_labels:
        if label[-3:] == 'vel':
            if label[-7] == '1':
                mouse_1_vel_labels.append(label)
            else:
                mouse_2_vel_labels.append(label)
        else:
            if label[-3] == '1':
                mouse_1_pos_labels.append(label)
            else:
                mouse_2_pos_labels.append(label)


    mouse_1_pos = n.zeros((len(mouse_1_pos_labels),len(trial)))
    for i,l in enumerate(mouse_1_pos_labels): mouse_1_pos[i]=trial[l]
    mouse_2_pos = n.zeros((len(mouse_2_pos_labels),len(trial)))
    for i,l in enumerate(mouse_2_pos_labels): mouse_2_pos[i]=trial[l]
    mouse_1_vel = n.zeros((len(mouse_1_vel_labels),len(trial)))
    for i,l in enumerate(mouse_1_vel_labels): mouse_1_vel[i]=trial[l]
    mouse_2_vel = n.zeros((len(mouse_2_vel_labels),len(trial)))
    for i,l in enumerate(mouse_2_vel_labels): mouse_2_vel[i]=trial[l]
    # TODO how to normalize??
    trial_data = n.concatenate([mouse_1_pos, mouse_2_pos, mouse_1_vel, mouse_2_vel])
    if nan is not None:
        trial_data = n.nan_to_num(trial_data, nan=nan)
    
    trial_labels = n.concatenate([mouse_1_pos_labels, mouse_2_pos_labels, mouse_1_vel_labels, mouse_2_vel_labels])
    
    return trial_data, trial_labels

## Separate train, test and val sets

In [5]:
def split_sets(features_all,targets_all, chunk_size=500, split = 0.8, separate_vid_idx = None):
    data_len = features_all.shape[0]
    num_chunks = int(data_len // chunk_size)
    chunk_list = n.random.choice(range(num_chunks), size=num_chunks, replace=False)

    test_chunk_idx_bound = split*num_chunks

    features_train = []
    features_test = []
    targets_train = []
    targets_test = []
    
    if separate_vid_idx is not None:
        targets_separate = []
        features_separate = []

    for i in range(num_chunks):
        curr_chunk_idx = chunk_list[i]*chunk_size
        curr_chunk = features_all[curr_chunk_idx:curr_chunk_idx+chunk_size,:]
        curr_chunk_t = targets_all[curr_chunk_idx:curr_chunk_idx+chunk_size]
#         print(curr_chunk_idx)
        if separate_vid_idx is not None and curr_chunk_idx+chunk_size > separate_vid_idx[0] and curr_chunk_idx < separate_vid_idx[1]:
#                 print(curr_chunk_idx, separate_vid_idx[0])
#                 print(curr_chunk_idx+chunk_size, separate_vid_idx[1])
                targets_separate.append(curr_chunk_t)
                features_separate.append(curr_chunk)
        elif i < test_chunk_idx_bound:
#             print("train!!")
            features_train.append(curr_chunk)
            targets_train.append(curr_chunk_t)
        else:
#             print('test')
            features_test.append(curr_chunk)
            targets_test.append(curr_chunk_t)
        
    features_train = n.concatenate(features_train, axis=0)
    features_test = n.concatenate(features_test, axis=0)
    
    targets_test = n.concatenate(targets_test)
    targets_train = n.concatenate(targets_train)
    
    if separate_vid_idx is None:
        return features_train, features_test, targets_train, targets_test
    else:
        features_separate = n.concatenate(features_separate, axis=0)
        targets_separate = n.concatenate(targets_separate)
        return features_train, features_test, features_separate,\
                targets_train, targets_test, targets_separate

def str_to_int(targets, mapping = None):
    categories = n.unique(targets)
    N_categories = len(categories)
    if mapping is None:
        mapping = {}
        i = 0
        for c in categories:
            mapping[c] = i
            i += 1
    targets_int = n.array([mapping[s] for s in targets], dtype=int)
    
    return targets_int, mapping
    

In [6]:
def init_weights(m):
    if type(m) == nn.Linear:
        torch.nn.init.xavier_uniform_(m.weight)
        m.bias.data.fill_(0.01)

class MLP():
    def __init__(self, architecture):
        self.architecture = architecture
        
        self.D_in = self.architecture['D_in']
        self.D_out = self.architecture['D_out']
        self.hidden_dims = self.architecture['hidden_dims']
        
        self.layers = []
        prev_dim = self.architecture['D_in']
        for dim in self.architecture['hidden_dims']:
            self.layers += [torch.nn.Linear(prev_dim, dim),
                           torch.nn.ReLU()]
            prev_dim = dim
        self.layers += [torch.nn.Linear(prev_dim, self.D_out)]
        self.model = nn.Sequential(*self.layers)
        
        self.trackers = {}

    def init_weights(self):
        self.model.apply(init_weights)
   
    def start_trackers(self,track, reset_trackers=True):
        if reset_trackers: self.trackers = {}
        for t in track:
            self.trackers[t] = []
    
    def track(self):
        for variable in self.trackers.keys():
            self.trackers[variable].append(copy.deepcopy(getattr(self, variable)))
        
    def learn(self,learning, training_set, test_set, reset_trackers=True, verbose=True):
        
        #self.init_weights()
        
        # set up variables
        N_batch = learning['N_batch']
        N_epochs = learning['N_epochs']
        loss_fn = learning['loss_fn']
        print_interval = learning['print_interval']
        track = learning['track']
        learning_rate = learning['learning_rate']
        
        self.learning = learning
        
        optimizer = learning['optimizer'](self.model.parameters(), learning_rate)
        self.start_trackers(track, reset_trackers)
        
        # load data
        x_train, y_train = training_set
        x_test, y_test = test_set

        N_training = len(y_train)
        N_test = len(y_test)
    
        self.t = 0
        tic = time.time()
        end = False
        for self.epoch_idx in range(N_epochs):
            if verbose: print("### EPOCH {:2d} ###".format(self.epoch_idx))
                
            # randomize batches
            indices = n.random.choice(range(N_training), N_training, False)
            num_batches = len(indices) // N_batch + 1

            for self.batch_idx in range(num_batches):
                # load batch
                b_idx = self.batch_idx
                x_train_batch = x_train[indices[b_idx*N_batch :(b_idx+1)*N_batch]]
                y_train_batch = y_train[indices[b_idx*N_batch : (b_idx+1)*N_batch]]

                
                # predict, loss and learn
                y_train_batch_pred = self.model(x_train_batch)
                loss = loss_fn(y_train_batch_pred, y_train_batch)
                self.train_loss = loss.item()
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()

                self.t += 1

                if self.batch_idx % print_interval == 0:
                    
#                     y_train_pred = self.model(x_train)
#                     self.train_loss = loss_fn(y_train_pred, y_train).item()
#                     pred_labels = n.argmax(y_train_pred.detach().numpy(),axis=1)
#                     true_labels = y_train.detach().numpy()
#                     correct_preds = n.array(pred_labels == true_labels, n.int)
                    self.train_frac_correct = 0# n.mean(correct_preds)
                    
                    y_test_pred = self.model(x_test)
                    self.test_loss = loss_fn(y_test_pred, y_test).item()
                    pred_labels = n.argmax(y_test_pred.detach().numpy(),axis=1)
                    true_labels = y_test.detach().numpy()
                    correct_preds = n.array(pred_labels == true_labels, n.int)
                    self.test_frac_correct = n.mean(correct_preds)

                    toc = time.time()
                    delta = toc - tic
                    tic = toc
                    print("Time: {:4.2f}, Batch {:3d}, Train Loss (for batch): {:4.2f}, Test Loss: {:4.2f}, Test Correct Frac: {:.3f}".format(\
                                       delta, self.batch_idx, self.train_loss, self.test_loss, self.test_frac_correct))
            
                    self.track()

In [7]:
def plot_net_results(net, x_test, y_test):
    plt.plot(net.trackers['t'],net.trackers['train_loss'], label='train')
    plt.plot(net.trackers['t'],net.trackers['test_loss'], label='test')
    plt.title("Cross Entropy Loss through training")
    plt.legend()
    plt.ylim(0,10)
    plt.show()
    
    plt.plot(net.trackers['t'], net.trackers['test_frac_correct'])
    plt.title("Fraction of correct labels on test set through training")
    plt.ylim(0.5,1)
    plt.show()
    
    max_perf_ind = n.argmax(net.trackers['test_frac_correct'])
    min_loss_ind = n.argmin(net.trackers['test_loss'])
    max_perf_model = net.trackers['model'][max_perf_ind]
    
    prediction_test = max_perf_model(x_test)
    pred = n.argmax(prediction_test.detach().numpy(),axis=1)
    true = y_test.detach().numpy()
    confmat = confusion_matrix(true, pred, normalize='true')
    f, ax = plt.subplots(figsize=(10,10))
    m = ax.matshow(confmat, cmap='Blues', vmin=0,  vmax=1)
    ax.set_xlabel("Predicted Label")
    ax.set_ylabel("True Label")
    ax.set_xticks(list(range(len(categories))))
    ax.set_xticklabels(categories, rotation=45)
    ax.set_yticks(list(range(len(categories))))
    ax.set_yticklabels(categories, rotation=45)
    f.colorbar(m)
    
    plt.show()

In [8]:

class LSTM_net(nn.Module):
    def __init__(self, input_dim, hidden_dim,output_dim, n_layers_LSTM, dropout_prob):
        super(LSTM_net, self).__init__()
        self.hidden_dim = hidden_dim
        self.input_dim = input_dim
        self.output_dim = output_dim
        self.n_layers = n_layers_LSTM
        
        self.lstm = nn.LSTM(input_size = input_dim, hidden_size = hidden_dim, 
                            num_layers  = n_layers_LSTM, dropout = dropout_prob,batch_first=True)
        self.dropout = nn.Dropout(dropout_prob)
        self.fc = nn.Linear(hidden_dim, output_dim)
#         self.out_func = nn.Softmax(dim=1)
    def forward(self, x, hidden=None):
        batch_size = x.size(0)
#         x = x.long()
#         print(x.shape)
        if hidden is None:
            lstm_out, hidden = self.lstm(x)
        else: 
            lstm_out, hidden = self.lstm(x,hidden)
        lstm_out = lstm_out.contiguous().view(-1, self.hidden_dim)
        
        out = self.dropout(lstm_out)
        out = self.fc(out)
        out = out.view(batch_size, self.output_dim, -1)[:,:,-1]
        
        return out, hidden
    def init_hidden(self, batch_size):
        weight = next(self.parameters()).data
        hidden = (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().to(device),
                      weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().to(device))
        return hidden

In [9]:
def extract_data(datas, n_window, n_stride, test_vids=[3,4],batch_size=32, shuffle=True):
    datapoints = []
    labels = []

    trial_bounds = []

    trial_idx = 0
    idx = 0
    
    for data in datas:  
        trial_bounds.append(idx)
        for i in range(int(data.shape[1]/n_stride)):
            if (i*n_stride + n_window) < data.shape[1]:
                datapoints.append(data[:,i*n_stride : i*n_stride + n_window])
                new_labels = [all_data[trial_keys[trial_idx]]['ann'][i*n_stride + int(n_window/2)]]
                new_labels = [target_map[l] for l in new_labels]
                labels.append(new_labels)
                idx += 1
        trial_idx += 1
    trial_bounds.append(idx)  
    
    Xs = n.stack(datapoints)
    Ys = n.stack(labels)
    num_features = Xs.shape[1]
    
    Xs_train = []; Ys_train = []; Xs_test = []; Ys_test = []
    for i in range(len(trial_bounds)-1):
        if i in test_vids:
            Xs_test.append(Xs[trial_bounds[i]:trial_bounds[i+1]-1])
            Ys_test.append(Ys[trial_bounds[i]:trial_bounds[i+1]-1])
        else:
            Xs_train.append(Xs[trial_bounds[i]:trial_bounds[i+1]-1])
            Ys_train.append(Ys[trial_bounds[i]:trial_bounds[i+1]-1])
#     print(Xs_train[0].shape)
    Xs_train = n.concatenate(Xs_train,axis=0).reshape(-1, n_window, num_features)
    Ys_train = n.concatenate(Ys_train,axis=0).reshape(-1)
    Xs_test = n.concatenate(Xs_test,axis=0).reshape(-1, n_window, num_features)
    Ys_test = n.concatenate(Ys_test,axis=0).reshape(-1)
    
    
    train_data = torch.utils.data.TensorDataset(torch.from_numpy(Xs_train), torch.from_numpy(Ys_train))
    test_data = torch.utils.data.TensorDataset(torch.from_numpy(Xs_test), torch.from_numpy(Ys_test))

    train_loader = torch.utils.data.DataLoader(train_data, shuffle=shuffle, batch_size=batch_size)
    test_loader = torch.utils.data.DataLoader(test_data, shuffle=shuffle, batch_size=batch_size)
    
    return train_data, train_loader, test_data, test_loader

In [10]:
def test_model(params, datas, device, random_val):
    train_data, train_loader, val_data, val_loader \
            = extract_data(datas, params['n_windows'], params['n_stride'],
                        batch_size=params['batch_size'],shuffle=params['shuffle'])
        
    model = LSTM_net(params['input_dim'], params['hidden_dim'], params['output_dim'], params['n_layers_LSTM'], params['dropout_prob'])
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=params['lr'])
    model.to(device)
    model.train()

    params['val_losses'] = []
    params['val_fracs'] = []
    
    counter = 0 
    tic = time.time()
    for i in range(params['N_epochs']):
        h = model.init_hidden(params['batch_size'])
        for inputs, labels in train_loader:
            if inputs.shape[0] != params['batch_size']: continue
            counter += 1
            h = tuple([e.data for e in h])
            inputs, labels = inputs.to(device), labels.to(device)
            model.zero_grad()
            output, h = model(inputs.float(), (h[0].float(), h[1].float()))
            loss = criterion(output.squeeze(), labels[:].long())
            loss.backward()
            nn.utils.clip_grad_norm_(model.parameters(), params['clip'])
            optimizer.step()
                        
        toc = time.time() - tic
        tic = time.time()
        
        if counter%params['print_every'] == 0:
            val_h = model.init_hidden(params['batch_size'])
            val_losses = []
            model.eval()
            fracs = []
            for inp, lab in val_loader:
                if inp.shape[0] != params['batch_size']: continue
                val_h = tuple([each.data for each in val_h])
                inp, lab = inp.to(device), lab.to(device)
                out, val_h = model(inp.float(), [val_h[0].float(), val_h[1].float()])
                val_loss = criterion(out.squeeze(), lab[:].long())
                val_losses.append(val_loss.item())
                
                pred_labels = n.argmax(out.cpu().detach().numpy(),axis=1)
                true_labels = lab.cpu().detach().numpy()
                correct_preds = n.array(pred_labels == true_labels, n.int)
                fracs.append(n.mean(correct_preds))
                
            frac_correct_val = (n.mean(fracs))
            params['val_losses'].append(n.mean(val_losses))
            params['val_fracs'].append(frac_correct_val)
            model.train()
            print("\tEpoch: {}/{}; {:.2f} sec...".format(i+1, params['N_epochs'],toc),
                  "\tLoss: {:.3f}...".format(loss.item()),
                  "\tVal Loss: {:.3f}".format(n.mean(val_losses)),
                  "\tVal Frac: {:.4f}".format(frac_correct_val))
            if n.mean(val_losses) <= params['valid_loss_min']:
                torch.save(model.state_dict(), './state_dict' + str(random_val) + '.pt')
                print('\tValidation loss decreased ({:.3f} --> {:.3f}).  Saving model ...'.format(params['valid_loss_min'],n.mean(val_losses)))
                params['valid_loss_min'] = n.mean(val_losses)
            if frac_correct_val >= params['valid_frac_max']:
                params['valid_frac_max'] = frac_correct_val
                params['pred_labels_best'] = pred_labels.copy()
                params['true_labels_best'] = true_labels.copy()

                
    return params, model
                
        

## Run


In [11]:
# Load data into a structured array
data_root = 'C:/Users/Neuropixel/AH-EN'
dlc_dir = 'postprocessedXYCoordinates'
ann_dir = 'manualannotations'
all_data = load_data(data_root, dlc_dir, ann_dir)

# Choose which position labels we care about
feature_labels = all_data['Female1'].dtype.names[2:]

# Calculate velocity and preprocess/scale/normalize data
trial_keys = list(all_data.keys())
datas = []
for key in trial_keys:
    datas.append(normalize_trial(all_data[key], feature_labels)[0])
features_all = n.concatenate(datas, axis=1).T

# Format category labels
targets_all = n.concatenate([all_data[key]['ann'] for key in trial_keys]).T
targets_int, target_map = str_to_int(targets_all)
categories = target_map.keys()
N_categories = len(categories)
input_dim = datas[0].shape[0]

In [12]:
device = torch.device("cuda")

In [13]:
params_original = {
    'n_windows'      : 50,
    'n_stride'       : 20,
    'shuffle'        : True,
    'dropout_prob'   : 0.3,
    'batch_size'     : 32,
    'hidden_dim'     : 50,
    'n_layers_LSTM'  : 1, 
    'output_dim'     : N_categories,
    'input_dim'      : input_dim,
    'lr'             : 0.005,
    'N_epochs'       : 1000,
    'print_every'    : 5,
    'clip'           : 5,
    'valid_loss_min' : n.Inf,
    'valid_frac_max' : 0,
}


In [14]:
sweeps = {
#     'n_stride'        : [5,50],
#     'dropout_prob'    : [0, 0.1, 0.5],
#     'batch_size'      : [8, 64, 128],
    'hidden_dim'      : [10, 20, 100],
    'n_layers_LSTM'   : [2, 3, 5],
    'lr'              : [0.001, 0.01],
    'n_windows'       : [1, 20, 50,100, 250],
}
# add shuffle
results = []

In [15]:
for key in sweeps.keys():
    for param_val in sweeps[key]:
        n.random.seed(2358)
        torch.manual_seed(2358)
        params = copy.copy(params_original)
        params[key] = param_val
        
        print("\n###########\nTesting with ", key, ": ", param_val)
        
        params_result, model = test_model(params, datas, device, int(n.random.random()*1e9))
        
        results.append((copy.deepcopy(params_result), copy.deepcopy(model)))
        
        print("Completed: ", key, ": ", param_val, "\n\tloss: {:.6f} \n\tfrac:{:.6f}".format(params_result['valid_loss_min'], params_result['valid_frac_max']))


###########
Testing with  hidden_dim :  10


C:\Users\Neuropixel\.conda\envs\en\lib\site-packages\torch\nn\modules\rnn.py:58: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.3 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


	Epoch: 5/1000; 1.50 sec... 	Step: 1215... 	Loss: 1.600... 	Val Loss: 1.037 	Val Frac: 0.7426
	Validation loss decreased (inf --> 1.037).  Saving model ...
	Epoch: 10/1000; 1.39 sec... 	Step: 2430... 	Loss: 1.135... 	Val Loss: 0.987 	Val Frac: 0.7268
	Validation loss decreased (1.037 --> 0.987).  Saving model ...
	Epoch: 15/1000; 1.52 sec... 	Step: 3645... 	Loss: 0.879... 	Val Loss: 0.819 	Val Frac: 0.7515
	Validation loss decreased (0.987 --> 0.819).  Saving model ...
	Epoch: 20/1000; 1.40 sec... 	Step: 4860... 	Loss: 1.326... 	Val Loss: 1.235 	Val Frac: 0.7096
	Epoch: 25/1000; 1.54 sec... 	Step: 6075... 	Loss: 1.077... 	Val Loss: 0.841 	Val Frac: 0.7386
	Epoch: 30/1000; 1.47 sec... 	Step: 7290... 	Loss: 1.358... 	Val Loss: 1.212 	Val Frac: 0.7228
	Epoch: 35/1000; 1.70 sec... 	Step: 8505... 	Loss: 1.428... 	Val Loss: 0.904 	Val Frac: 0.7121
	Epoch: 40/1000; 1.54 sec... 	Step: 9720... 	Loss: 0.861... 	Val Loss: 0.770 	Val Frac: 0.7390
	Validation loss decreased (0.819 --> 0.770).  Savi

	Epoch: 400/1000; 1.50 sec... 	Step: 97200... 	Loss: 0.907... 	Val Loss: 0.718 	Val Frac: 0.7669
	Epoch: 405/1000; 1.57 sec... 	Step: 98415... 	Loss: 0.749... 	Val Loss: 0.710 	Val Frac: 0.7507
	Epoch: 410/1000; 1.42 sec... 	Step: 99630... 	Loss: 1.492... 	Val Loss: 0.747 	Val Frac: 0.7456
	Epoch: 415/1000; 1.46 sec... 	Step: 100845... 	Loss: 0.908... 	Val Loss: 0.689 	Val Frac: 0.7853
	Epoch: 420/1000; 1.53 sec... 	Step: 102060... 	Loss: 0.854... 	Val Loss: 0.718 	Val Frac: 0.7665
	Epoch: 425/1000; 1.52 sec... 	Step: 103275... 	Loss: 0.666... 	Val Loss: 0.712 	Val Frac: 0.7835
	Epoch: 430/1000; 1.50 sec... 	Step: 104490... 	Loss: 1.398... 	Val Loss: 0.789 	Val Frac: 0.7471
	Epoch: 435/1000; 1.55 sec... 	Step: 105705... 	Loss: 1.179... 	Val Loss: 0.738 	Val Frac: 0.7743
	Epoch: 440/1000; 1.60 sec... 	Step: 106920... 	Loss: 1.161... 	Val Loss: 0.688 	Val Frac: 0.7871
	Epoch: 445/1000; 1.52 sec... 	Step: 108135... 	Loss: 0.967... 	Val Loss: 0.704 	Val Frac: 0.7761
	Epoch: 450/1000; 1.55 

	Epoch: 815/1000; 1.62 sec... 	Step: 198045... 	Loss: 0.793... 	Val Loss: 0.670 	Val Frac: 0.7820
	Epoch: 820/1000; 1.70 sec... 	Step: 199260... 	Loss: 0.701... 	Val Loss: 0.661 	Val Frac: 0.7912
	Epoch: 825/1000; 1.45 sec... 	Step: 200475... 	Loss: 0.794... 	Val Loss: 0.646 	Val Frac: 0.7912
	Epoch: 830/1000; 1.52 sec... 	Step: 201690... 	Loss: 0.533... 	Val Loss: 0.617 	Val Frac: 0.8000
	Validation loss decreased (0.621 --> 0.617).  Saving model ...
	Epoch: 835/1000; 1.59 sec... 	Step: 202905... 	Loss: 0.958... 	Val Loss: 0.648 	Val Frac: 0.7974
	Epoch: 840/1000; 1.47 sec... 	Step: 204120... 	Loss: 0.942... 	Val Loss: 0.697 	Val Frac: 0.7831
	Epoch: 845/1000; 1.37 sec... 	Step: 205335... 	Loss: 0.849... 	Val Loss: 0.725 	Val Frac: 0.7820
	Epoch: 850/1000; 1.69 sec... 	Step: 206550... 	Loss: 0.930... 	Val Loss: 0.642 	Val Frac: 0.8004
	Epoch: 855/1000; 1.45 sec... 	Step: 207765... 	Loss: 0.921... 	Val Loss: 0.833 	Val Frac: 0.7669
	Epoch: 860/1000; 1.43 sec... 	Step: 208980... 	Loss: 

...

	Epoch: 350/1000; 1.64 sec... 	Step: 84700... 	Loss: 1.467... 	Val Loss: 1.156 	Val Frac: 0.0729
	Epoch: 355/1000; 1.71 sec... 	Step: 85910... 	Loss: 1.278... 	Val Loss: 1.146 	Val Frac: 0.0986
	Epoch: 360/1000; 1.62 sec... 	Step: 87120... 	Loss: 1.255... 	Val Loss: 1.148 	Val Frac: 0.1097
	Epoch: 365/1000; 1.72 sec... 	Step: 88330... 	Loss: 1.116... 	Val Loss: 1.146 	Val Frac: 0.1109
	Epoch: 370/1000; 1.58 sec... 	Step: 89540... 	Loss: 1.190... 	Val Loss: 1.142 	Val Frac: 0.1112
	Epoch: 375/1000; 1.87 sec... 	Step: 90750... 	Loss: 1.292... 	Val Loss: 1.139 	Val Frac: 0.1008
	Epoch: 380/1000; 1.65 sec... 	Step: 91960... 	Loss: 1.125... 	Val Loss: 1.153 	Val Frac: 0.1008
	Epoch: 385/1000; 1.59 sec... 	Step: 93170... 	Loss: 1.430... 	Val Loss: 1.149 	Val Frac: 0.0975
	Epoch: 390/1000; 1.56 sec... 	Step: 94380... 	Loss: 1.444... 	Val Loss: 1.189 	Val Frac: 0.0696
	Epoch: 395/1000; 1.67 sec... 	Step: 95590... 	Loss: 1.333... 	Val Loss: 1.157 	Val Frac: 0.0547
	Epoch: 400/1000; 1.76 sec... 

	Epoch: 750/1000; 1.67 sec... 	Step: 181500... 	Loss: 1.533... 	Val Loss: 1.120 	Val Frac: 0.0934
	Epoch: 755/1000; 1.82 sec... 	Step: 182710... 	Loss: 1.132... 	Val Loss: 1.119 	Val Frac: 0.0856
	Epoch: 760/1000; 1.66 sec... 	Step: 183920... 	Loss: 1.239... 	Val Loss: 1.128 	Val Frac: 0.0923
	Epoch: 765/1000; 1.63 sec... 	Step: 185130... 	Loss: 1.326... 	Val Loss: 1.129 	Val Frac: 0.0822
	Epoch: 770/1000; 1.75 sec... 	Step: 186340... 	Loss: 1.149... 	Val Loss: 1.134 	Val Frac: 0.0725
	Epoch: 775/1000; 1.82 sec... 	Step: 187550... 	Loss: 1.223... 	Val Loss: 1.125 	Val Frac: 0.0804
	Epoch: 780/1000; 1.64 sec... 	Step: 188760... 	Loss: 0.911... 	Val Loss: 1.123 	Val Frac: 0.0666
	Epoch: 785/1000; 1.84 sec... 	Step: 189970... 	Loss: 1.231... 	Val Loss: 1.108 	Val Frac: 0.0722
	Epoch: 790/1000; 1.67 sec... 	Step: 191180... 	Loss: 1.170... 	Val Loss: 1.105 	Val Frac: 0.0744
	Epoch: 795/1000; 1.69 sec... 	Step: 192390... 	Loss: 1.193... 	Val Loss: 1.148 	Val Frac: 0.0744
	Epoch: 800/1000; 1.

	Epoch: 130/1000; 3.44 sec... 	Step: 31330... 	Loss: 0.751... 	Val Loss: 0.715 	Val Frac: 0.7630
	Epoch: 135/1000; 3.33 sec... 	Step: 32535... 	Loss: 0.852... 	Val Loss: 0.658 	Val Frac: 0.7909
	Validation loss decreased (0.660 --> 0.658).  Saving model ...
	Epoch: 140/1000; 3.54 sec... 	Step: 33740... 	Loss: 0.952... 	Val Loss: 0.742 	Val Frac: 0.7597
	Epoch: 145/1000; 3.41 sec... 	Step: 34945... 	Loss: 1.024... 	Val Loss: 0.677 	Val Frac: 0.7667
	Epoch: 150/1000; 3.39 sec... 	Step: 36150... 	Loss: 0.600... 	Val Loss: 0.750 	Val Frac: 0.7556
	Epoch: 155/1000; 3.39 sec... 	Step: 37355... 	Loss: 0.658... 	Val Loss: 0.675 	Val Frac: 0.7775
	Epoch: 160/1000; 3.58 sec... 	Step: 38560... 	Loss: 0.543... 	Val Loss: 0.644 	Val Frac: 0.7842
	Validation loss decreased (0.658 --> 0.644).  Saving model ...
	Epoch: 165/1000; 3.40 sec... 	Step: 39765... 	Loss: 1.101... 	Val Loss: 0.667 	Val Frac: 0.7805
	Epoch: 170/1000; 3.41 sec... 	Step: 40970... 	Loss: 0.832... 	Val Loss: 0.669 	Val Frac: 0.7760

	Epoch: 530/1000; 3.49 sec... 	Step: 127730... 	Loss: 0.714... 	Val Loss: 0.620 	Val Frac: 0.7928
	Validation loss decreased (0.621 --> 0.620).  Saving model ...
	Epoch: 535/1000; 3.58 sec... 	Step: 128935... 	Loss: 0.652... 	Val Loss: 0.635 	Val Frac: 0.7917
	Epoch: 540/1000; 3.46 sec... 	Step: 130140... 	Loss: 0.850... 	Val Loss: 0.651 	Val Frac: 0.7861
	Epoch: 545/1000; 3.45 sec... 	Step: 131345... 	Loss: 0.784... 	Val Loss: 0.632 	Val Frac: 0.7928
	Epoch: 550/1000; 3.51 sec... 	Step: 132550... 	Loss: 0.850... 	Val Loss: 0.630 	Val Frac: 0.7827
	Epoch: 555/1000; 3.42 sec... 	Step: 133755... 	Loss: 1.168... 	Val Loss: 0.610 	Val Frac: 0.8036
	Validation loss decreased (0.620 --> 0.610).  Saving model ...
	Epoch: 560/1000; 3.56 sec... 	Step: 134960... 	Loss: 0.785... 	Val Loss: 0.646 	Val Frac: 0.7868
	Epoch: 565/1000; 3.55 sec... 	Step: 136165... 	Loss: 0.949... 	Val Loss: 0.640 	Val Frac: 0.7943
	Epoch: 570/1000; 3.52 sec... 	Step: 137370... 	Loss: 0.850... 	Val Loss: 0.669 	Val Fra

	Epoch: 945/1000; 3.48 sec... 	Step: 227745... 	Loss: 0.893... 	Val Loss: 0.644 	Val Frac: 0.7790
	Epoch: 950/1000; 3.51 sec... 	Step: 228950... 	Loss: 0.465... 	Val Loss: 0.636 	Val Frac: 0.7839
	Epoch: 955/1000; 3.43 sec... 	Step: 230155... 	Loss: 0.552... 	Val Loss: 0.636 	Val Frac: 0.7868
	Epoch: 960/1000; 3.51 sec... 	Step: 231360... 	Loss: 0.961... 	Val Loss: 0.624 	Val Frac: 0.7965
	Epoch: 965/1000; 3.45 sec... 	Step: 232565... 	Loss: 0.665... 	Val Loss: 0.626 	Val Frac: 0.7961
	Epoch: 970/1000; 3.35 sec... 	Step: 233770... 	Loss: 0.685... 	Val Loss: 0.649 	Val Frac: 0.7842
	Epoch: 975/1000; 3.50 sec... 	Step: 234975... 	Loss: 0.466... 	Val Loss: 0.652 	Val Frac: 0.7865
	Epoch: 980/1000; 3.41 sec... 	Step: 236180... 	Loss: 0.728... 	Val Loss: 0.609 	Val Frac: 0.8058
	Validation loss decreased (0.610 --> 0.609).  Saving model ...
	Epoch: 985/1000; 3.49 sec... 	Step: 237385... 	Loss: 0.806... 	Val Loss: 0.651 	Val Frac: 0.7824
	Epoch: 990/1000; 3.41 sec... 	Step: 238590... 	Loss: 

In [20]:
for result in results: 
    print("########\nresult loss: {:.6f} \nresult frac:{:.6f}".format(result[0]['valid_loss_min'], result[0]['valid_frac_max']))
    print(result[0])
    

########
result loss: 0.617172 
result frac:0.801838
{'n_windows': 50, 'n_stride': 20, 'shuffle': True, 'dropout_prob': 0.3, 'batch_size': 32, 'hidden_dim': 10, 'n_layers_LSTM': 1, 'output_dim': 8, 'input_dim': 40, 'lr': 0.005, 'N_epochs': 1000, 'print_every': 5, 'clip': 5, 'valid_loss_min': 0.6171717030160567, 'valid_frac_max': 0.8018382352941177, 'val_losses': [1.0371554816470427, 0.9868090201826657, 0.8191421533332152, 1.2353495829245624, 0.8405808357631459, 1.2118288362727445, 0.9042030025930966, 0.7698272445622613, 1.0254128364955677, 1.0769603785346535, 0.9242537239018609, 0.9490161913282731, 0.8094653206713059, 1.1695227756219753, 1.4808050323935116, 0.8298907511374529, 0.9736046454485725, 0.7815301930203157, 0.7439049420987859, 0.9772262187565074, 0.7770780458169825, 0.9203896964297575, 1.0271419784601996, 0.8474342451376073, 0.7344633074367748, 0.7323199777042164, 0.8473169319769915, 0.7586980237680323, 0.8205578453400556, 0.7870850892627941, 0.9184956424376544, 0.715805661327

In [21]:
import pickle

In [22]:
filehandler = open(b"test.res","wb")
pickle.dump(result[-1],filehandler)